__Importing the necessary libraries__
_we need to import the all the libraries we need to execute this tasks_


In [ ]:
import csv
import json
import xml.etree.ElementTree as ET
from collections import defaultdict
from pony.orm import Database, PrimaryKey, Required, Optional, db_session, commit, set_sql_debug

**Assinging varibles to the file path!**

In [15]:
xml_file_path = "user_data_23_4.xml"
csv_file_path = "user_data_23_4.csv"
json_file_path = "user_data_23_4.json"
txt_file_path = "user_data_23_4.txt"

**Reading and Parsing the csv file**:
_We define a function to read and parse the csv file and return the content of the csv. list [] to store the csv data that has been read and parsed_

In [16]:
def read_csv(csv_file_path):
# Initialize an empty list to store the data in csv file.
    data_csv = []
# Open the CSV file
    with open(csv_file_path, 'r') as csv_file:
        # Create a CSV reader object
        csv_reader = csv.DictReader(csv_file)
# Iterate over each row in the CSV file
        for row in csv_reader:
# Append each row to the data list
            data_csv.append(row)
# Return the collected data
    return data_csv
# Specify the path to the CSV file
csv_file_data= "user_data_23_4.csv"
 # Call the function to read the CSV file and store the data
csv_data = read_csv(csv_file_path)


**Reading and Parsing the json file** :
We define a function to read and parse the json file and return the content of the json file. 
 use a list [] and a  dictionary {} to store the json data that has been read and parsed.

In [17]:
def read_json(json_file_path):
# Initialize an empty list to store the data in json file
    data_json = []
# Open the JSON file
    with open(json_file_path, 'r') as json_file:
# Load JSON data from the file
        json_data = json.load(json_file)
# Iterate over each row in the JSON data
        for row in json_data:
# Extract debt information from the row
            debt = row.get('debt', '0')
# Convert debt to a dictionary if it's a string
            if type(debt) == str:
                debt = {"amount": debt, "time_period_years": 0}
# Create a dictionary for each row with specified keys
            json_dict = {
                'firstName': row['firstName'],
                'lastName': row['lastName'],
                'age': row['age'],
                'iban': row['iban'],
                'credit_card_number': row['credit_card_number'],
                'credit_card_security_code': row['credit_card_security_code'],
                'credit_card_start_date': row['credit_card_start_date'],
                'credit_card_end_date': row['credit_card_end_date'],
                'address_main': row['address_main'],
                'address_city': row['address_city'],
                'address_postcode': row['address_postcode'],
                'debit': debt,
            }
# Append the dictionary to the list
            data_json.append(json_dict)
# Return the collected list of dictionaries
    return data_json
# Specify the path to the JSON file
json_file_path = "user_data_23_4.json"

# Call the function to read the JSON file and store the data
json_data = read_json(json_file_path)

**Read and Parsing the XML file!** :


In [18]:
def read_xml(xml_file_path):
# Parse the XML file
    tree = ET.parse(xml_file_path)
    root = tree.getroot()
# Initialize an empty list to store the data
    data_xml = []
# Iterate through each 'user' element in the XML
    for user_element in root.findall('user'):
# Convert attributes of 'user' element into a dictionary
        xml_dict = dict(user_element.attrib)
# Check if 'dependants' attribute exists and replace with '0' if empty
        if not xml_dict.get('dependants'):
            xml_dict['dependants'] = '0'
# Append user dictionary to data list
        data_xml.append(xml_dict)
# Return the collected data
    return data_xml
# Specify the path to the XML file
xml_file_path = "user_data_23_4.xml"
# Call the function to read the XML file and store the data
xml_data = read_xml(xml_file_path)

**Merging All the Files(CSV, JSON and XML)!**


In [19]:
def all_merged_datasets(csv_data, json_data, xml_data):
    m_data = defaultdict(dict)
# Merged CSV data
    for record in csv_data:
        fn = (record.get('firstName') or record.get('First Name'),
              record.get('lastName') or record.get('Second Name'))
#fixing the firstName, LastName and Second Name discrepancies in the files as they appear to be different in each files.
        m_data[fn].update({
            'firstName': record.get('firstName'),
            'lastName': record.get('lastName'),
            'age': record.get('age') or record.get('Age (Years)'),
            'sex': record.get('sex') or record.get('Sex'),
            **{f: n for f, n in record.items() if f not in ['firstName', 'First Name', 'lastName', 'Second Name', 'age', 'Age (Years)', 'sex', 'Sex']}
        })
# Merged JSON data
    for record in json_data:
        fn = (record.get('firstName'), record.get('lastName'))
        m_data[fn].update({
            'firstName': record.get('firstName'),
            'lastName': record.get('lastName'),
            'age': record.get('age'),
            'sex': record.get('sex'),
            **{f: n for f, n in record.items() if f not in ['firstName', 'lastName', 'age', 'sex']}
        })
# Merged XML data
    for record in xml_data:
        fn = (record.get('firstName'), record.get('lastName'))
        m_data[fn].update({
            'firstName': record.get('firstName'),
            'lastName': record.get('lastName'),
            'age': record.get('age'),
            'sex': record.get('sex') or record.get('Sex'),
            **{f: n for f, n in record.items() if f not in ['firstName', 'lastName', 'age', 'sex', 'Sex']}
        })

# Convert merged records to a list
    merged_documents = list(m_data.values())
    return merged_documents

documents_unified = all_merged_datasets(csv_data, json_data, xml_data)

**Creating a DataBase with Pony.orm**

In [20]:
# Initialize a database object
db = Database()
# Define an entity named Userdata234unified representing a database table
class DocumentsUnified(db.Entity):
# Define a primary key column named 'id' that automatically increments
    id = PrimaryKey(int, auto=True)
# Define required columns for first name, last name
    firstName = Required(str)
    lastName = Required(str)
# Define optional columns for other attributes like age, sex, etc.
    age = Optional(int)
    sex = Optional(str)
    Vehicle_Make = Optional(str)
    Vehicle_Model = Optional(str)
    Vehicle_Year = Optional(int)
    Vehicle_Type = Optional(str)
    iban = Optional(str)
    credit_card_number = Optional(str)
    credit_card_security_code = Optional(str)
    credit_card_start_date = Optional(str)
    credit_card_end_date = Optional(str)
    address_main = Optional(str)
    address_city = Optional(str)
    address_postcode = Optional(str)
    retired = Optional(bool)
    dependants = Required(int)
    marital_status = Optional(str)
    salary = Optional(float)
    pension = Optional(float)
    company = Optional(str)
    commute_distance = Optional(float)

# Establish a connection to the MySQL database
# 'bind' method connects the database with specified parameters
db.bind(provider='mysql', host='europa.ashley.work', user='secbi87no', passwd='iE93F2@8EhM@1zhD&u9M@K', db='secbi87no', ssl_ca='./ca-cert.pem')
# Enable SQL debugging, which logs SQL queries executed
set_sql_debug(True)
# Generate mapping between Python classes and database tables
# 'generate_mapping' creates necessary tables if they don't exist
db.generate_mapping(create_tables=True)

GET NEW CONNECTION
RELEASE CONNECTION
GET CONNECTION FROM THE LOCAL POOL
SET foreign_key_checks = 0
SELECT `documentsunified`.`id`, `documentsunified`.`firstname`, `documentsunified`.`lastname`, `documentsunified`.`age`, `documentsunified`.`sex`, `documentsunified`.`vehicle_make`, `documentsunified`.`vehicle_model`, `documentsunified`.`vehicle_year`, `documentsunified`.`vehicle_type`, `documentsunified`.`iban`, `documentsunified`.`credit_card_number`, `documentsunified`.`credit_card_security_code`, `documentsunified`.`credit_card_start_date`, `documentsunified`.`credit_card_end_date`, `documentsunified`.`address_main`, `documentsunified`.`address_city`, `documentsunified`.`address_postcode`, `documentsunified`.`retired`, `documentsunified`.`dependants`, `documentsunified`.`marital_status`, `documentsunified`.`salary`, `documentsunified`.`pension`, `documentsunified`.`company`, `documentsunified`.`commute_distance`
FROM `documentsunified` `documentsunified`
WHERE 0 = 1

COMMIT
SET forei

**Matching the records in the combined/merged files with that of the database created!**

In [23]:
# Decorator to define a database session
def insert_all_documents(combined_documents):
    try:
# Iterate through each item in the combined data
        for item in combined_documents:
# Create a new instance of DocumentsUnified entity and populate its attributes
            DocumentsUnified(
                firstName=item['firstName'],
                lastName=item['lastName'],
                age=int(item['age']),
                sex=item['sex'],
                Vehicle_Make=item['Vehicle Make'],
                Vehicle_Model=item['Vehicle Model'],
                Vehicle_Year=int(item['Vehicle Year']),
                Vehicle_Type=item['Vehicle Type'],
                iban=item['iban'],
                credit_card_number=item['credit_card_number'],
                credit_card_security_code=item['credit_card_security_code'],
                credit_card_start_date=item['credit_card_start_date'],
                credit_card_end_date=item['credit_card_end_date'],
                address_main=item['address_main'],
                address_city=item['address_city'],
                address_postcode=item['address_postcode'],
                retired=item['retired'].lower() == 'false',  # Convert to boolean
                dependants=int(item['dependants']),
                marital_status=item['marital_status'],
                salary=float(item['salary']),
                pension=float(item['pension']),
                company=item['company'],
                commute_distance=float(item['commute_distance'])
            )
# Commit the changes to the database
        commit()
        print("Data insertion successful.")
    except Exception as e:
# Handle any exceptions that occur during data insertion
        print(f"Error during data insertion: {e}")

# Call the insert_all_data function with the unified data
insert_all_documents(documents_unified)

INSERT INTO `documentsunified` (`firstname`, `lastname`, `age`, `sex`, `vehicle_make`, `vehicle_model`, `vehicle_year`, `vehicle_type`, `iban`, `credit_card_number`, `credit_card_security_code`, `credit_card_start_date`, `credit_card_end_date`, `address_main`, `address_city`, `address_postcode`, `retired`, `dependants`, `marital_status`, `salary`, `pension`, `company`, `commute_distance`) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
['Leon', 'Smith', 48, 'Male', 'Mercedes-Benz', 'Astro Cargo', 2009, 'Pickup', 'GB05DRJP98363742599825', '2292406027303835', '750', '09/18', '01/22', '0 Shaw plains', "O'Neillshire", 'DY0 3SG', True, 2, 'married or civil partner', 65911.0, 0.0, 'Gilbert, Williams and Quinn', 15.68]

INSERT INTO `documentsunified` (`firstname`, `lastname`, `age`, `sex`, `vehicle_make`, `vehicle_model`, `vehicle_year`, `vehicle_type`, `iban`, `credit_card_number`, `credit_card_security_code`, `credit_card_start_date`, `cre

In [24]:
txt_file_path = "user_data_23_4.txt"
# Open the file using 'with open' statement
with open(txt_file_path, 'r') as file:
# Read the contents of the file
    file_contents = file.read()
# Print the contents of the file
print(file_contents)

"Valerie Ellis e-mailed in overnight (Full details in Ticket #1839). During account creation something went wrong and their security code is wrong on their billing information. Bank is rejecting any payment until it's corrected. They're not sure what happened, but said to try "762". Can you please action this and try re-bill the client? Let me know if there's any further issues with it. Thanks"
"Congratulations on the promotion West! We wouldn't have survived through the pandemic without you and your team. As a token of our appreciation, we've given you a Â£2100 salary bump. This will take effect as of next month's payroll. At Williams-Wheeler we value the care and work you put in. See you on Monday Charlie, enjoy!"
"Happy Birthday Mr Charlie Short! You're 52 today. Our latest offers will be sure to get you into the party spirit!"
"Hannah, the pension policy has changed slightly since the meeting last week. I know you've just finished putting through all the changes, but I need you to 

**Querying my sql to generate information of a customers from the text file**

In [28]:
# Define a function to update the credit card security code
def update_credit_card_security_code():
    try:
# Retrieve the record for user with ID 625 from the Userdata234unified entity
        valerie_ellis = DocumentsUnified.get(id=625)
# Check if the record exists
        if valerie_ellis:
# Update the credit card security code for the user
            valerie_ellis.credit_card_security_code = '726'
# Commit the changes to the database
            commit()
            print("Credit card security code updated successfully.")
        else:
# Print a message if the user is not found
            print("User not found.")
    except Exception as e:
# Handle any exceptions that occur during the update process
        print(f"Error during update: {e}")

# Call the update_credit_card_security_code function to execute the update operation
update_credit_card_security_code()

UPDATE `documentsunified`
SET `credit_card_security_code` = %s
WHERE `id` = %s
  AND `firstname` = %s
  AND `lastname` = %s
  AND `age` = %s
  AND `sex` = %s
  AND `vehicle_make` = %s
  AND `vehicle_model` = %s
  AND `vehicle_year` = %s
  AND `vehicle_type` = %s
  AND `iban` = %s
  AND `credit_card_number` = %s
  AND `credit_card_security_code` = %s
  AND `credit_card_start_date` = %s
  AND `credit_card_end_date` = %s
  AND `address_main` = %s
  AND `address_city` = %s
  AND `address_postcode` = %s
  AND `retired` = %s
  AND `dependants` = %s
  AND `marital_status` = %s
  AND `company` = %s
['726', 625, 'Valerie', 'Ellis', 55, 'Female', 'Toyota', 'Impala', 2001, 'Sedan', 'GB95UFRA89299972043981', '4460612841317016', '276', '01/22', '03/27', '65 Frank rue', 'New Leanne', 'NP57 8ZL', True, 2, 'married or civil partner', 'Carter PLC']

COMMIT
Credit card security code updated successfully.


In [35]:
# Define a function to update Charlie West's record
def change_charlie_west_record():
    try:
# Retrieve the record for Charlie West from the Userdata234unified entity
        update_charlie_record = DocumentsUnified.get(id=62)
# Check if the record exists
        if update_record:
# Increase Charlie West's salary by 2100
            update_charlie_record.salary += 2100
# Commit the changes to the database
            commit()
            print("Increment successful.")
        else:
# Print a message if the user is not found
            print("User not found.")
    except Exception as e:
# Handle any exceptions that occur during the update process
        print(f"Error during update: {e}")
# Call the update_charlie_west_record function to execute the update operation
change_charlie_west_record()

UPDATE `documentsunified`
SET `pension` = %s
WHERE `id` = %s
  AND `firstname` = %s
  AND `lastname` = %s
  AND `age` = %s
  AND `sex` = %s
  AND `vehicle_make` = %s
  AND `vehicle_model` = %s
  AND `vehicle_year` = %s
  AND `vehicle_type` = %s
  AND `iban` = %s
  AND `credit_card_number` = %s
  AND `credit_card_security_code` = %s
  AND `credit_card_start_date` = %s
  AND `credit_card_end_date` = %s
  AND `address_main` = %s
  AND `address_city` = %s
  AND `address_postcode` = %s
  AND `retired` = %s
  AND `dependants` = %s
  AND `marital_status` = %s
  AND `company` = %s
[0.0, 986, 'Louis', 'Leonard', 37, 'Male', 'Hyundai', 'Silverado (Classic) 1500 HD Crew Cab', 2006, 'Van/Minivan', 'GB83GJEO03803888176852', '4300668320666867085', '6083', '03/17', '12/25', '4 Henry points', 'West Rhysfurt', 'NW0V 5QZ', True, 2, 'married or civil partner', 'Gordon, Carey and Edwards']

UPDATE `documentsunified`
SET `salary` = %s
WHERE `id` = %s
  AND `firstname` = %s
  AND `lastname` = %s
  AND `age`

In [42]:
def update_martin_file():   
# Retrieve the record for Martin Christian with ID 986 from the database
    update_martin_record = DocumentsUnified.get(id='986')
    try:      
# Check if the record exists
        if update_martin_record:
# Existing pension amount stored for Mr. Martin Christian
            existing_pension = update_record.pension
# Adding another 0.15% to the existing pension of Martin Christian
            new_percentage = 0.15
            new_amount = existing_pension * (new_percentage / 100)
# Updating the pension amount of Martin Christian in the Database          
            update_record.pension += new_amount
# Commit the changes to the database
            commit()
            print("Increment Pension successful.")
        else:
# Print a message if the user is not found
            print("User not found.")
    except Exception as e:
# Handle any exceptions that occur during the update process
        print(f"Error during update: {e}")
# Call the update_charlie_west_record function to execute the update operation
update_martin_file()

UPDATE `documentsunified`
SET `pension` = %s
WHERE `id` = %s
  AND `firstname` = %s
  AND `lastname` = %s
  AND `age` = %s
  AND `sex` = %s
  AND `vehicle_make` = %s
  AND `vehicle_model` = %s
  AND `vehicle_year` = %s
  AND `vehicle_type` = %s
  AND `iban` = %s
  AND `credit_card_number` = %s
  AND `credit_card_security_code` = %s
  AND `credit_card_start_date` = %s
  AND `credit_card_end_date` = %s
  AND `address_main` = %s
  AND `address_city` = %s
  AND `address_postcode` = %s
  AND `retired` = %s
  AND `dependants` = %s
  AND `marital_status` = %s
  AND `company` = %s
[0.0, 986, 'Louis', 'Leonard', 37, 'Male', 'Hyundai', 'Silverado (Classic) 1500 HD Crew Cab', 2006, 'Van/Minivan', 'GB83GJEO03803888176852', '4300668320666867085', '6083', '03/17', '12/25', '4 Henry points', 'West Rhysfurt', 'NW0V 5QZ', True, 2, 'married or civil partner', 'Gordon, Carey and Edwards']

COMMIT
Increment Pension successful.
